## 5.3 売上予測モデルを活用したデータドリブン販促（小売りチェーン）

### 5.3.1 事例説明

今後どうなるかという近未来分析も実施し、先を見通しながらデータドリブンな販促活動を実施する。  
そのためには、売上予測モデルを構築する必要がある。  
売上予測モデルを構築したら、どのプランがよさそうかどうかを検討するため、mROIを施策プラン案の評価で利用する。  
$mROI=\frac{増分売上 - 増分コスト}{増分コスト}$

プランA：チラシ配布と値引きの両方を実施（現行プラン）  
プランB：チラシ配布のみ実施  
プランC：値引きのみ実施  
プランD：チラシ配布も値引きもしない

各施策プラン案のmROIは、プランDに対する増分売上と増分コストを使って計算する。  
そのため、プランDのmROIは計算できない。

### 5.3.2 データセットと分析概要

予測モデルを構築するとき、どの数理モデルとどの説明変数で予測モデルを構築すればいいのか、ということを検討する。

#### 説明変数  
パターン１：説明変数x1からx4、休日フラグ（holiday）  
パターン２：説明変数x1からx4、休日フラグ（holiday）、曜日ダミー変数  
パターン３：説明変数x1からx4、休日フラグ（holiday）、三角関数特徴量  

#### 数理モデル
Ridge回帰モデル  
PLS回帰モデル

### 5.3.3 Pythonの実施例

#### ざっくりした流れ

1.準備  
2.予測モデルの作り方の検討  
3.予測モデルの構築  
4.各施策プラン案に対する売上予測  
5.各施策プラン案に対するmROI算定

#### ステップ１：準備

#### Code 5-32

In [1]:
# 必要なモジュールの読み込み

import numpy as np
import pandas as pd

from pmdarima.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

import optuna
from optuna.integration import OptunaSearchCV
from optuna.distributions import FloatDistribution
from optuna.distributions import IntDistribution

import warnings
warnings.simplefilter('ignore')

import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [12,9]
plt.rcParams['font.size'] = 14

##### Optunaの説明
このコードは、Optuna を使用してハイパーパラメータの最適化を行うための準備をしています。具体的には、OptunaSearchCV を使って機械学習モデルのハイパーパラメータを探索するための設定を行います。

##### 各インポートの説明
##### import optuna
- Optunaは自動ハイパーパラメータ最適化ライブラリです。
- optuna をインポートすることで、試行 (trial) や最適化 (study) の作成が可能になります。
##### from optuna.integration import OptunaSearchCV
- OptunaSearchCV は、Optuna を Scikit-learn の GridSearchCV のように使用 できる統合クラスです。
- sklearn.model_selection.GridSearchCV のように、ハイパーパラメータの探索を行うことができます。
#### from optuna.distributions import FloatDistribution
- 連続値のハイパーパラメータ（例: 学習率 0.01 〜 0.1）を定義するためのクラスです。
- 例: FloatDistribution(0.01, 0.1) は 0.01 から 0.1 の範囲の値を探索します。
#### from optuna.distributions import IntDistribution
- 整数値のハイパーパラメータ（例: 決定木の最大深さ 3 〜 10）を定義するためのクラスです。
- 例: IntDistribution(3, 10) は 3 から 10 の整数値を探索します。


#### cede 5-33

In [2]:
# 必要なデータセットの読み込み

dataset = 'chap5_2.csv'
df=pd.read_csv(dataset)

#### code-5-34

In [3]:
# パターン2の曜日のダミー変数（One-Hotエンコーディング）生成

# One-Hot エンコーディングのカテゴリとして、曜日をリストに格納

week_list = ['Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'Mon' ]

# インスタンス生成

enc = OneHotEncoder(                                          # エンコーディング対象として曜日リストを指定
    categories = [week_list], sparse_output=False)  # sparse → sparse_output  出力形式は疎行列ではなく密行列にする
# デフォルトでは 疎行列（sparse matrix, scipy.sparse） を返しますが、sparse_output=False を設定すると NumPy の密行列 になります。

# One-Hotエンコーディングの実施
# fit_transform メソッドは、指定された列（df[['day_of_the_week']]）を使って 
# One-Hot エンコーディングを行い、その結果を week_d に格納
# df[['day_of_the_week']] は、曜日情報を持つデータフレームの列

week_d = enc.fit_transform(df[['day_of_the_week']])

# データフレーム化
# エンコードされた week_d を Pandas のデータフレームに変換し、曜日名を列名として指定
# また、元のデータフレーム df のインデックスを保持するようにしている
# dtype='int' によって、出力されるダミー変数は整数型になる

df_week_d = pd.DataFrame(
    week_d,
    index=df.index,
    columns=week_list,
    dtype='int')

# 確認
print(df_week_d)

     Tue  Wed  Thu  Fri  Sat  Sun  Mon
0      0    0    0    0    0    0    1
1      1    0    0    0    0    0    0
2      0    1    0    0    0    0    0
3      0    0    1    0    0    0    0
4      0    0    0    1    0    0    0
..   ...  ...  ...  ...  ...  ...  ...
136    0    0    1    0    0    0    0
137    0    0    0    1    0    0    0
138    0    0    0    0    1    0    0
139    0    0    0    0    0    1    0
140    0    0    0    0    0    0    1

[141 rows x 7 columns]


In [4]:
df[['day_of_the_week']]

,day_of_the_week
0,Mon
1,Tue
2,Wed
3,Thu
4,Fri
...,...
136,Thu
137,Fri
138,Sat
139,Sun


In [5]:
df.head()

,date,day_of_the_week,y1,y21,y22,x1,x2,x3,x4,holiday
0,2016/6/6,Mon,818979,177,4627,18.3,0.0,0.006,0,0
1,2016/6/7,Tue,789332,164,4813,19.8,0.9,0.002,0,0
2,2016/6/8,Wed,793266,141,5626,22.8,0.5,0.002,0,0
3,2016/6/9,Thu,721415,157,4595,23.1,7.0,0.005,0,0
4,2016/6/10,Fri,796138,146,5453,22.2,0.0,0.003,0,0


#### code 5-35

In [6]:
# 元のデータセットに結合

df = pd.concat([df, df_week_d], axis=1)

#### code 5-36

#### コードの概要
- df_week_ft というデータフレームを作成（df と同じインデックスを持つ）
- fourier_terms_gen 関数で三角関数（sin, cos）の特徴量を作成
- 周期（seasonal=7, 1週間）を考慮し、3組の sin/cos 特徴量を追加
- 結果を df_week_ft に格納し、出力

In [7]:
# パターン3の三角関数特徴量の生成

# 空のデータフレーム生成

df_week_ft = pd.DataFrame()  # 空のデータフレームを作成
df_week_ft.index = df.index  # df（元データフレーム）と同じ時間軸を持つようにするためインデックスを引き継ぐ

# Fourier termsの生成関数

def fourier_terms_gen(X, seasonal, terms_num):

    # X：特徴量を追加するデータフレーム（ここでは df_week_ft）ここではdfとの結合は行わない
    # seasonal:周期
    # terms_num:Fourier termの数（sinとcosのセット数）

    for num in range(terms_num):
        num = num + 1  # rangeは0から始まるのでnumは1,2,3...
        # 列名を作成（例: "sin7_1", "cos7_1", "sin7_2", "cos7_2"）
        sin_colname = 'sin' + str(seasonal) + '_' + str(num)  
        cos_colname = 'cos' + str(seasonal) + '_' + str(num)

        # フーリエ級数　時間の流れに応じた周期的な変動を数値化できる
        X[sin_colname] = np.sin(num*2*np.pi*X.index/seasonal)
        X[cos_colname] = np.cos(num*2*np.pi*X.index/seasonal)

# 三角関数特徴量の生成
# seasonal=7（1週間の周期）
# terms_num=3（3組の sin/cos を追加）
# 結果：df_week_ft に以下の列が追加
# "sin7_1", "cos7_1","sin7_2", "cos7_2","sin7_3", "cos7_3"

fourier_terms_gen(df_week_ft, seasonal=7, terms_num=3)


print(df_week_ft)

           sin7_1    cos7_1        sin7_2    cos7_2        sin7_3    cos7_3
0    0.000000e+00  1.000000  0.000000e+00  1.000000  0.000000e+00  1.000000
1    7.818315e-01  0.623490  9.749279e-01 -0.222521  4.338837e-01 -0.900969
2    9.749279e-01 -0.222521 -4.338837e-01 -0.900969 -7.818315e-01  0.623490
3    4.338837e-01 -0.900969 -7.818315e-01  0.623490  9.749279e-01 -0.222521
4   -4.338837e-01 -0.900969  7.818315e-01  0.623490 -9.749279e-01 -0.222521
..            ...       ...           ...       ...           ...       ...
136  4.338837e-01 -0.900969 -7.818315e-01  0.623490  9.749279e-01 -0.222521
137 -4.338837e-01 -0.900969  7.818315e-01  0.623490 -9.749279e-01 -0.222521
138 -9.749279e-01 -0.222521  4.338837e-01 -0.900969  7.818315e-01  0.623490
139 -7.818315e-01  0.623490 -9.749279e-01 -0.222521 -4.338837e-01 -0.900969
140 -4.898587e-15  1.000000 -9.797174e-15  1.000000  1.372595e-14  1.000000

[141 rows x 6 columns]


#### フーリエ級数の基本式
$sin(\frac{2πnt}{T})$,  $cos(\frac{2πnt}{T})$
- T = seasonal（周期、ここでは 7）
- n = num（フーリエ次数）
- t = X.index（時間軸）  
これにより、時間の流れに応じた周期的な変動を数値化できる

#### code 5-37

In [8]:
# 元のデータセットに結合

df = pd.concat([df, df_week_ft], axis=1)

In [9]:
df.head()

,date,day_of_the_week,y1,y21,y22,x1,x2,x3,x4,holiday,...,Fri,Sat,Sun,Mon,sin7_1,cos7_1,sin7_2,cos7_2,sin7_3,cos7_3
0,2016/6/6,Mon,818979,177,4627,18.3,0.0,0.006,0,0,...,0,0,0,1,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
1,2016/6/7,Tue,789332,164,4813,19.8,0.9,0.002,0,0,...,0,0,0,0,0.781831,0.623490,0.974928,-0.222521,0.433884,-0.900969
2,2016/6/8,Wed,793266,141,5626,22.8,0.5,0.002,0,0,...,0,0,0,0,0.974928,-0.222521,-0.433884,-0.900969,-0.781831,0.623490
3,2016/6/9,Thu,721415,157,4595,23.1,7.0,0.005,0,0,...,0,0,0,0,0.433884,-0.900969,-0.781831,0.623490,0.974928,-0.222521
4,2016/6/10,Fri,796138,146,5453,22.2,0.0,0.003,0,0,...,1,0,0,0,-0.433884,-0.900969,0.781831,0.623490,-0.974928,-0.222521


#### code 5-38

In [10]:
# データセットを学習データとテストデータ（直近21日間）に分割

target_length = 21  # テストデータ直近21日間　その前は学習データ

df_train, df_test = train_test_split(
    df, test_size=target_length)

#### ステップ2：予測モデルの作り方の検討

数理モデル（2）  
-Ridge回帰  
-PLS回帰

変数の選択パターン（3）  
パターン１：説明変数x1からx4、[休日フラグ]（holiday)  
パターン２：説明変数x1からx4、[休日フラグ]（holiday)、曜日ダミー変数  
パターン３：説明変数x1からx4、[休日フラグ]（holiday)、三角関数特徴量

6つの予測モデルをRMSE、MAE、MAPEの３つの指標で評価する。

#### code 5-39

パターン1の「説明変数x1からx4、『休日フラグ変数』（holiday）」を使い、Ridge回帰とPLS回帰で予測モデルを構築し、テストデータを評価する。  
目的変数yと説明変数Xの設定をする。  
説明変数の選択は「True（選択）」と「False（除外）」で表現されたリストを使う。

In [11]:
df.head(3)

,date,day_of_the_week,y1,y21,y22,x1,x2,x3,x4,holiday,...,Fri,Sat,Sun,Mon,sin7_1,cos7_1,sin7_2,cos7_2,sin7_3,cos7_3
0,2016/6/6,Mon,818979,177,4627,18.3,0.0,0.006,0,0,...,0,0,0,1,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
1,2016/6/7,Tue,789332,164,4813,19.8,0.9,0.002,0,0,...,0,0,0,0,0.781831,0.623490,0.974928,-0.222521,0.433884,-0.900969
2,2016/6/8,Wed,793266,141,5626,22.8,0.5,0.002,0,0,...,0,0,0,0,0.974928,-0.222521,-0.433884,-0.900969,-0.781831,0.623490


In [12]:
# パターン１｜目的変数yと説明変数X （説明変数x1からx4、[休日フラグ]（holiday)）

# 変数選択リスト（True:dfの列のうち説明変数として残す）

x_list = [
    False,False,False,False,False,                    # date～y22(5)
    True,True,True,True,True,                         # x1～holiday(5)
    False,False,False,False,False,False,False,  # day_of_the_week(7)
    False,False,False,False,False,False            # Fourier terms(6)
]

print(df[df.columns[x_list]])

# 学習データ  code5-38で分離している

y_train = df_train.y1
X_train = df_train[df.columns[x_list]]

# テストデータ  code5-38で分離している

y_test = df_test.y1
X_test = df_test[df.columns[x_list]]

       x1    x2     x3    x4  holiday
0    18.3   0.0  0.006     0        0
1    19.8   0.9  0.002     0        0
2    22.8   0.5  0.002     0        0
3    23.1   7.0  0.005     0        0
4    22.2   0.0  0.003     0        0
..    ...   ...    ...   ...      ...
136  20.8   0.0  0.002     0        0
137  17.8   0.5  0.007     0        0
138  17.7  82.9  0.229  7500        1
139  16.9  76.5  0.225  7500        1
140  17.3  10.8  0.010     0        0

[141 rows x 5 columns]


Ridge回帰の学習。  
Ridge回帰のパイパーパラメータの探索から始める。

#### code 5-40

In [13]:
# ハイパーパラメータ探索

# Ridge回帰（L2正則化付き線形回帰）のインスタンス生成
# ここでは alpha（正則化パラメータ）の値を最適化する。

regressor = Ridge()

# 探索するハイパーパラメータ範囲の設定
# alpha の値を [0.001, 10^10] の範囲で探索する。
# FloatDistribution(0.001, 1e10) は Optuna の 連続値の探索範囲 を定義する。

params = {
    'alpha':
        FloatDistribution(0.001, 1e10)
}

# ハイパーパラメータ探索の設定
# OptunaSearchCV は Optuna を用いたグリッドサーチ。
# estimator=regressor: Ridge回帰を最適化する。
# param_distributions=params: 探索するパラメータは alpha のみ。
# n_trials=1000: 1000回の試行（異なる alpha の組み合わせ）を行う。
# cv=TimeSeriesSplit(): 時系列データ向けの交差検証（時系列データの分割方法）。
# random_state=123: 乱数シードを固定（再現性を確保）。

optuna_search = OptunaSearchCV(
    estimator=regressor,
    param_distributions=params,
    n_trials=1000,
    cv=TimeSeriesSplit(),
    random_state=123)

# 探索実施
# optuna.logging.set_verbosity(optuna.logging.WARNING): ログ出力を抑える（警告のみ表示）。
# optuna_search.fit(X_train, y_train): X_train と y_train を使って最適な alpha を探索。

optuna.logging.set_verbosity(optuna.logging.WARNING)
optuna_search.fit(X_train, y_train)

# 探索結果

print(optuna_search.best_params_)

{'alpha': 3031229722.737783}


この求めた入パラメータの値を使いRidge回帰で予測モデルを学習する。

#### code 5-41

In [14]:
# 最適ハイパーパラメータで学習

# 最適なハイパーパラメータを設定
# optuna_search.best_params_ は Optuna で見つかった最適なハイパーパラメータ を辞書形式で取得する。
# set_params(**optuna_search.best_params_) は、リッジ回帰モデル (regressor) のパラメータを最適なものに変更する。
# ** は 辞書のアンパック（展開） を行う Python の演算子です。
# set_params(**optuna_search.best_params_) のように使うと、辞書のキーと値を関数の引数として展開できます。
# つまり、**optuna_search.best_params_ は alpha=3031229722.737783 のように展開される。
# ridge_p1 という新しい変数に 最適なパラメータを持つ Ridge モデル を代入。

ridge_p1 = regressor.set_params(**optuna_search.best_params_)

# 学習データで学習
# X_train（学習データの説明変数）と y_train（学習データの目的変数）を使って、最適な alpha を適用したリッジ回帰モデルを訓練する。
# fit() を実行すると、モデルが X_train と y_train に最適な回帰係数（重み）を学習する。

ridge_p1.fit(X_train, y_train)

Ridge(alpha=3031229722.737783)

テストデータで評価する

#### code 5-42

In [15]:
# テストのデータで精度評価

# 予測
train_pred = ridge_p1.predict(X_train)
test_pred = ridge_p1.predict(X_test)

# 精度指標（テストデータ）
print('RMSE:\n', np.sqrt(mean_squared_error(y_test, test_pred)))
print('MAE:\n', mean_absolute_error(y_test, test_pred))
print('MAPE:\n', mean_absolute_percentage_error(y_test, test_pred))

RMSE:
 440563.1171193202
MAE:
 259485.55946852308
MAPE:
 0.1274521135025654


次に、PLS回帰の学習。PLS回帰のハイパーパラメータの探索から始める。

#### code 5-43

In [16]:
# ハイパーパラメータ探索

# PLS回帰のインスタンス生成
# PLSRegression() は 部分最小二乗回帰（PLS回帰） を実装するための関数。
# PLS回帰は多重共線性がある場合や高次元データに適した回帰手法。

regressor = PLSRegression()

# 探索するハイパーパラメータ範囲の設定
# n_components: PLS回帰で使用する潜在変数（成分数）。
# IntDistribution(1, len(X_train.columns)) は 1から特徴量数までの整数値の範囲 で n_components を探索。

params = {
    'n_components':
        IntDistribution(1, len(X_train.columns))
}

# ハイパーパラメータ探索の設定
# OptunaSearchCV を使用して Optuna によるハイパーパラメータ探索 を実施。
# n_trials=1000 で 最大1000回の試行 を行う。
# cv=TimeSeriesSplit() により 時系列データに適した交差検証 を使用。
# random_state=123 で 結果を再現可能 にする。

optuna_search = OptunaSearchCV(
    estimator=regressor,
    param_distributions=params,
    n_trials=1000,
    cv=TimeSeriesSplit(),
    random_state=123)

# 探索実施
# optuna.logging.set_verbosity(optuna.logging.WARNING) で Optunaの出力を抑制。
# optuna_search.fit(X_train, y_train) で ハイパーパラメータ探索を実行。

optuna.logging.set_verbosity(optuna.logging.WARNING)
optuna_search.fit(X_train, y_train)

# 探索結果
# optuna_search.best_params_ で 最適な n_components を取得。

print(optuna_search.best_params_)

{'n_components': 5}


この求めたハイパーパラメータの値を使い、PLS回帰で予測モデルを学習する。

#### code 5-44

In [17]:
# 最適ハイパーパラメータで学習

# 最適なハイパーパラメータを設定
PLS_p1 = regressor.set_params(**optuna_search.best_params_)

# 学習データで学習
PLS_p1.fit(X_train, y_train)

PLSRegression(n_components=5)

テストデータで評価する。

#### code 5-45

In [18]:
# テストデータで精度評価

# 予測
train_pred = PLS_p1.predict(X_train)
test_pred = PLS_p1.predict(X_test)

# 精度指標（テストデータ）
print('RMSE:\n', np.sqrt(mean_squared_error(y_test, test_pred)))
print('MAE:\n', mean_absolute_error(y_test, test_pred))
print('MAPE:\n', mean_absolute_percentage_error(y_test, test_pred))


RMSE:
 26138.600169956586
MAE:
 16289.280312699493
MAPE:
 0.008169169500944758


先ほどのRidge回帰と比べ、評価指標の値がよくなっている。  
同じような処理を変数の組み合わせパターンの数だけ繰り返すことになるため、処理を関数かする。

#### code 5-46

In [19]:
# 予測モデルを学習しその結果を返す関数

'''
関数名：
    prediction_model_generation
引数：
    regressor  学習器
    y_train  学習データの目的変数y
    X_train  学習データの説明変数X
    y_test  テストデータの目的変数y
    X_test  テストデータの説明変数X
戻値：
    train_pred  学習データの予測結果
    test_pred  テストデータの予測結果
    rmse  RMSE（精度評価指標）
    mae  MAE（精度評価指標）
    mape  MAPE（精度評価指標）
'''

def prediction_model_generation(regressor, y_train, X_train, y_test, X_test):

    # PLS回帰のインスタンス生成
    regressor = regressor

    # 探索するパイパーパラメータ範囲の設定
    if type(regressor) == PLSRegression:
        params = {
            'n_components':
                IntDistribution(1, len(X_train.columns))      # X_train.columns → X.columns
        }
    elif type(regressor)  == Ridge:
        params = {
            'alpha':FloatDistribution(0.001, 1e10)
        }
    else:
        return

    # ハイパーパラメータ探索の設定
    optuna_search = OptunaSearchCV(
        estimator=regressor,
        param_distributions=params,
        n_trials=1000,
        cv=TimeSeriesSplit(),
        random_state=123)

    # 探索実施
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    optuna_search.fit(X_train, y_train)

    # 最適なハイパーパラメータを設定」
    model = regressor.set_params(**optuna_search.best_params_)

    # 学習データで学習
    model.fit(X_train, y_train)

    # 予測
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)

    # 評価指標（テストデータ）
    rmse = np.sqrt(mean_squared_error(y_test, test_pred))
    mae = mean_absolute_error(y_test, test_pred)
    mape = mean_absolute_percentage_error(y_test, test_pred)

    # 戻値
    return train_pred,test_pred,rmse,mae,mape
                                      

この関数を使い、残りのパターンの評価をしていく。  
パターン2の「説明変数x1からx4、『休日フラグ（holiday）』、曜日ダミー変数」。  
目的変数yと説明変数yと説明変数Xの設定をする。

#### code 5-47

In [20]:
# パターン２｜目的変数yと説明変数X

# 変数選択リスト（True：変数として残す）
x_list = [
    False,False,False,False,False,
    True,True,True,True,True,                  # x1,x2,x3,x4.holiday
    True,True,True,True,True,True,True,  # day_of_the_week
    False,False,False,False,False,False     # Forurier terms
]

print(df[df.columns[x_list]])

# 学習データ
y_train = df_train.y1
X_train = df_train[df.columns[x_list]]

# テストデータ
y_test = df_test.y1
X_test = df_test[df.columns[x_list]]

       x1    x2     x3    x4  holiday  Tue  Wed  Thu  Fri  Sat  Sun  Mon
0    18.3   0.0  0.006     0        0    0    0    0    0    0    0    1
1    19.8   0.9  0.002     0        0    1    0    0    0    0    0    0
2    22.8   0.5  0.002     0        0    0    1    0    0    0    0    0
3    23.1   7.0  0.005     0        0    0    0    1    0    0    0    0
4    22.2   0.0  0.003     0        0    0    0    0    1    0    0    0
..    ...   ...    ...   ...      ...  ...  ...  ...  ...  ...  ...  ...
136  20.8   0.0  0.002     0        0    0    0    1    0    0    0    0
137  17.8   0.5  0.007     0        0    0    0    0    1    0    0    0
138  17.7  82.9  0.229  7500        1    0    0    0    0    1    0    0
139  16.9  76.5  0.225  7500        1    0    0    0    0    0    1    0
140  17.3  10.8  0.010     0        0    0    0    0    0    0    0    1

[141 rows x 12 columns]


#### code 5-48

In [21]:
# パターン2|Ridge回帰

# 学習と評価の実施
train_pred, test_pred, rmse, mae,mape=prediction_model_generation(
    Ridge(),
    y_train, X_train,
    y_test, X_test)

# 精度指標（テストデータ）
print('RMSE:\n', rmse)
print('MAE:\n', mae)
print('MAPE\n', mape)

RMSE:
 440563.1168484656
MAE:
 259485.55897910334
MAPE
 0.1274521130315328


PLS回帰で予測モデルを構築し、テストデータで評価する。

#### code 5-49

In [22]:
# パターン2|PLS回帰

# 学習と評価の実施
train_pred, test_pred, rmse, mae, mape=prediction_model_generation(
    PLSRegression(),
    y_train, X_train,
    y_test, X_test)

# 精度指標（テストデータ）
print('RMSE:\n', rmse)
print('MAE:\n', mae)
print('MAPE\n', mape)

RMSE:
 25811.19440188819
MAE:
 14744.53673676704
MAPE
 0.006547528395274088


これまでの中で一番、評価指標の値がよくなっている。  
続いて、パターン3の「説明変数x1からx4、『休日フラグ』（holiday）、三角関数特徴量」である。  
目的変数ｙと説明変数Xの設定をする。

#### code 5-50

In [23]:
# パターン3|目的変数yと説明変数X

# 変数選択リスト（True:変数として残す）
x_list = [
    False,False,False,False,False,
    True,True,True,True,True,                  # x1,x2,x3,x4.holiday
    False,False,False,False,False,False,False,  # day_of_the_week
    True,True,True,True,True,True     # Forurier terms
]    

print(df[df.columns[x_list]])

# 学習データ
y_train = df_train.y1
X_train = df_train[df.columns[x_list]]

# テストデータ
y_test = df_test.y1
X_test = df_test[df.columns[x_list]]

       x1    x2     x3    x4  holiday        sin7_1    cos7_1        sin7_2  \
0    18.3   0.0  0.006     0        0  0.000000e+00  1.000000  0.000000e+00   
1    19.8   0.9  0.002     0        0  7.818315e-01  0.623490  9.749279e-01   
2    22.8   0.5  0.002     0        0  9.749279e-01 -0.222521 -4.338837e-01   
3    23.1   7.0  0.005     0        0  4.338837e-01 -0.900969 -7.818315e-01   
4    22.2   0.0  0.003     0        0 -4.338837e-01 -0.900969  7.818315e-01   
..    ...   ...    ...   ...      ...           ...       ...           ...   
136  20.8   0.0  0.002     0        0  4.338837e-01 -0.900969 -7.818315e-01   
137  17.8   0.5  0.007     0        0 -4.338837e-01 -0.900969  7.818315e-01   
138  17.7  82.9  0.229  7500        1 -9.749279e-01 -0.222521  4.338837e-01   
139  16.9  76.5  0.225  7500        1 -7.818315e-01  0.623490 -9.749279e-01   
140  17.3  10.8  0.010     0        0 -4.898587e-15  1.000000 -9.797174e-15   

       cos7_2        sin7_3    cos7_3  
0    1.0000

Ridge回帰で予測モデルを構築し、テストデータで評価する。

#### code 5-51

In [24]:
# パターン3|Ridge回帰

# 学習と評価の実施
train_pred, test_pred, rmse, mae, mape=prediction_model_generation(
    Ridge(),
    y_train,X_train,
    y_test,X_test)

# 精度指導（テストデータ）
print('RMSE:\n', rmse)
print('MAE:\n', mae)
print('MAPE\n', mape)

RMSE:
 440563.1161713291
MAE:
 259485.55775555442
MAPE
 0.12745211185395158


PLS回帰で予測モデルを構築し、テストデータで評価する。

#### code 5-52

In [25]:
# パターン3|PLS回帰

# 学習と評価の実施
train_pred, test_pred, rmse, mae, mape=prediction_model_generation(
    PLSRegression(),
    y_train, X_train,
    y_test, X_test)

# 精度指標（テストデータ）
print('RMSE:\n', rmse)
print('MAE:\n', mae)
print('MAPE\n', mape)

RMSE:
 26483.566069971443
MAE:
 15531.593241985227
MAPE
 0.007312407992935583


#### ステップ3:予測モデルの構築

売上予測モデルを全データを使って構築する。

#### code 5-53

In [26]:
# パターン2の説明変数を使いPLS回帰で構築

# 変数選択リスト（True:変数として残す）
x_list = [
    False,False,False,False,False,
    True,True,True,True,True,                  # x1,x2,x3,x4.holiday
    True,True,True,True,True,True,True,  # day_of_the_week
    False,False,False,False,False,False     # Forurier terms
]

# 学習データ（全データ）
y = df.y1
X = df[df.columns[x_list]]

# PLS回帰のインスタンス作成
regressor = PLSRegression()

# 探索するハイパーパラメータ範囲の設定
params = {
    'n_components':
        IntDistribution(1, len(X_train.columns))
}

# ハイパーパラメータ探索の設定
optuna_search = OptunaSearchCV(
    estimator=regressor,
    param_distributions=params,
    n_trials=1000,
    cv=TimeSeriesSplit(),
    random_state=123)

# 探索実施
optuna.logging.set_verbosity(optuna.logging.WARNING)
optuna_search.fit(X,y)

# 最適なハイパーパラメータを設定
PLS_p2 = regressor.set_params(**optuna_search.best_params_)

# 学習データで学習
PLS_p2.fit(X,y)

# 切片と回帰係数
print('切片:\n',PLS_p2.intercept_)
print('回帰係数:\n',PLS_p2.coef_)

切片:
 [1364223.53191489]
回帰係数:
 [[-1.20019630e+03 -1.40755426e+04  4.00361350e+06  1.27909891e+01
   6.84232402e+05  1.06392136e+04  9.37754757e+02 -3.10951443e+03
   4.75514915e+03  4.25219966e+01 -7.24156719e+03 -5.78452783e+03]]


#### code 5-54

In [27]:
# 構築したモデルの保存

# Python の標準ライブラリ pickle をインポートします。
# pickle は、Python オブジェクトをバイナリ形式でシリアライズ（保存）・デシリアライズ（読み込み）するためのモジュールです。

import pickle

# 保存するファイル名 'model.pkl' を変数 filename に格納しています。
# .pkl はピクル化（pickle で保存）したファイルによく使われる拡張子です。

filename = 'model.pkl'

# pickle.dump() を使って PLS_p2（PLS 回帰などのモデル）をファイル 'model.pkl' に保存します。
# open(filename, 'wb') は、filename という名前のファイルを バイナリ書き込み（'wb'）モード で開きます。
# pickle.dump() は、オブジェクト（ここでは PLS_p2）を シリアライズ（直列化） し、ファイルに保存します。

pickle.dump(PLS_p2, open(filename, 'wb'))

#### code 5-55

In [28]:
# 保存したモデルの読み込み

import pickle

filename = 'model.pkl'

# open(filename, 'rb')：ファイル 'model.pkl' を バイナリ読み込み（'rb'）モード で開く。
# pickle.load(...) を使って、保存されていた PLS モデル（または他のオブジェクト）を復元し、変数 PLS_p2 に格納。

PLS_p2 = pickle.load(open(filename, 'rb'))

In [29]:
PLS_p2.fit(X,y)
# 切片と回帰係数
print('切片:\n',PLS_p2.intercept_)
print('回帰係数:\n',PLS_p2.coef_)

切片:
 [1364223.53191489]
回帰係数:
 [[-1.20019630e+03 -1.40755426e+04  4.00361350e+06  1.27909891e+01
   6.84232402e+05  1.06392136e+04  9.37754757e+02 -3.10951443e+03
   4.75514915e+03  4.25219966e+01 -7.24156719e+03 -5.78452783e+03]]


#### ステップ4：各施策案に対する売上予測

ここでは、この角施策プラン案に対する1週間（7日間）のデータを使い、売上予測を実施する。施策プラン案は次の4つである。  
プランA：チラシ配布と値引きの両方を実施（現行プラン）  
プランB：チラシ配布のみ実施  
プランC：値引きのみ実施  
プランD：チラシ配布も値引きもしない  

各施策プラン案のデータセットを読み込む

#### code 5-56

In [30]:
# 各施策プラン案のデータセットの読み込み

dataset = 'chap5_3.csv'
df_plan = pd.read_csv(dataset)

print(df_plan)

     plan        date day_of_the_week    x1   x2   x3     x4  holiday
0   planA  2016/10/25             Tue  20.6  3.8  0.0      0        0
1   planA  2016/10/26             Wed  21.0  3.7  0.0      0        0
2   planA  2016/10/27             Thu  20.4  0.5  0.0      0        0
3   planA  2016/10/28             Fri  18.5  0.0  0.0      0        0
4   planA  2016/10/29             Sat  17.1  0.0  0.2  75000        1
5   planA  2016/10/30             Sun  16.7  0.0  0.2  75000        1
6   planA  2016/10/31             Mon  17.3  9.3  0.0      0        0
7   planB  2016/10/25             Tue  20.6  3.8  0.0      0        0
8   planB  2016/10/26             Wed  21.0  3.7  0.0      0        0
9   planB  2016/10/27             Thu  20.4  0.5  0.0      0        0
10  planB  2016/10/28             Fri  18.5  0.0  0.0      0        0
11  planB  2016/10/29             Sat  17.1  0.0  0.0  75000        1
12  planB  2016/10/30             Sun  16.7  0.0  0.0  75000        1
13  planB  2016/10/3

季節性を表現するために、曜日ダミー変数を作る。

#### code 5-57

In [31]:
# 曜日のダミー変数（One-Hotエンコーディング）生成

# 曜日リスト
week_list = ['Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'Mon']

# One-Hotエンコーディングの実施
week_d = enc.transform(df_plan[['day_of_the_week']])

# データフレーム化
df_week_d = pd.DataFrame(
    week_d,
    index=df_plan.index,
    columns=week_list,
    dtype='int')

# 元のデータセットに結合
df_plan = pd.concat([df_plan, df_week_d], axis=1)

各施策プラン案のコストのデータセットを読み込む。

#### code 5-58

In [32]:
# コストのデータセットの読み込み

dataset = 'chap5_4.csv'
df_cost = pd.read_csv(dataset, index_col='plan')

# 確認
print(df_cost)

          cost
plan          
planA  4650202
planB   750000
planC  3675508
planD        0


プランAの売上を予測する。まず、説明変数Xを設定する。

#### code 5-59

In [33]:
# 利用する説明変数Xの設定

plan = 'planA'
X_plan = df_plan[df_plan.plan == plan].iloc[:,3:]

print(X_plan)

     x1   x2   x3     x4  holiday  Tue  Wed  Thu  Fri  Sat  Sun  Mon
0  20.6  3.8  0.0      0        0    1    0    0    0    0    0    0
1  21.0  3.7  0.0      0        0    0    1    0    0    0    0    0
2  20.4  0.5  0.0      0        0    0    0    1    0    0    0    0
3  18.5  0.0  0.0      0        0    0    0    0    1    0    0    0
4  17.1  0.0  0.2  75000        1    0    0    0    0    1    0    0
5  16.7  0.0  0.2  75000        1    0    0    0    0    0    1    0
6  17.3  9.3  0.0      0        0    0    0    0    0    0    0    1


この変数のデータを使って売上予測をする。

#### code 5-60

In [34]:
# 売上予測の実施

pred = PLS_p2.predict(X_plan)  # 予測の実施
sum_pred = np.sum(pred)  # 予測の合計

# 確認
print('daily:\n', pred)
print('total:\n', sum_pred)


daily:
 [ 749200.75863255  740426.7755165   782141.36058054  799324.16845687
 3240571.0979947  3233767.0873246   659322.18040797]
total:
 10204753.428913742


予測した1週間の売上の合計値をデータフレームに順次格納していく。  
そのためには、格納するデータフレームを作る必要がある。

#### code 5-61

In [35]:
# 予測結果をデータフレームへ格納

# データフレームの生成
pred_tbl = pd.DataFrame(
    index = df_cost.index,
    columns = ['predicted'])

# 予測結果の格納
pred_tbl.loc[plan, 'predicted'] = sum_pred

print(pred_tbl)

             predicted
plan                  
planA  10204753.428914
planB              NaN
planC              NaN
planD              NaN


プランAにだけ予測値が代入されている。  
異常の一連の処理を順次実施していくのは面倒なため、この一連の処理を関数化する。  
この関数を使い、残りの施策プラン案の予測などを実施していく。


#### code 5-62

In [38]:
#　予測関数

'''
関数名：
        predicted_sales
引数
        df_plan  計画のデータセット
        plan  プラン指定
        pred_tbl  予測結果を格納するデータフレーム
'''
def predicted_sales(df_plan, plan, pred_tbl):
    X_plan = df_plan[df_plan.plan == plan].iloc[:,3:]  # 説明変数Xの設定
    pred = PLS_p2.predict(X_plan)  # 予測の実施
    sum_pred = np.sum(pred)  # 予測の合計
    pred_tbl.loc[plan,'predicted'] = sum_pred  # 結果格納

残りの施策プラン案に対して、売上予測を実施し、結果を先ほど作ったデータフレームに順次格納していく。

#### code 5-63

In [39]:
# 残りの施策プラン案（planB, planC, planD)の予測実施

plan_list = ['planB', 'planC', 'planD']

for plan in plan_list:
    predicted_sales(df_plan, plan, pred_tbl)

print(pred_tbl)

             predicted
plan                  
planA  10204753.428914
planB   8603308.029109
planC   8286105.068745
planD   6684659.668939


どの施策プラン案が最もコスパがよいのか。

#### ステップ5:各施策案に対するmROI算定

#### code 5-64

In [40]:
# 各施策プラン案に対するマーケティングROIの算定

mROI = ((pred_tbl-pred_tbl.loc["planD"]).predicted - df_cost.cost) / df_cost.cost

print(mROI)

plan
planA   -0.243023
planB    1.558198
planC   -0.564293
planD         NaN
dtype: object
